In [1]:
import pandas as pd
import numpy as np
import os
from pandasql import sqldf
from scipy import stats
import time
import networkx as nx

In [ ]:
dataset_loc = '...'
bnf_df = pd.read_csv(dataset_loc)

bnf_list = [[]] # Insert list of prescription pairs to test directionality for eg. for pairs with RR > 1 in either/both directions

dir_result = []

# For each pair, find patients who are prescribed with each prescription in the pair at least once
# and add columns for which prescrption appears first
for bnf1, bnf2 in bnf_list:
    q = """
    SELECT MIN(a.patid), MIN(a.eventdate), MIN(a.bnfchapter), MAX(b.patid), MIN(b.eventdate), MIN(b.bnfchapter),
    MIN(a.eventdate)<MIN(b.eventdate) as D1first, MIN(a.eventdate)>MIN(b.eventdate) as D2first
    FROM bnf_dx AS a
    JOIN bnf_dx AS b
    ON a.patid = b.patid
    WHERE a.bnfchapter = {}
    AND b.bnfchapter = {}
    GROUP BY a.patid
    LIMIT 10000000;
    """.format(bnf1,bnf2)
    pairdf = sqldf(q)
    D1first = pairdf['D1first'].sum() # Sum number of times D1 prescribed first
    D2first = pairdf['D2first'].sum() # Sum number of times D2 prescribed first
    total = len(pairdf) # Total number of patients with both prescriptions
    D1pval = stats.binom_test(D1first, n=total, p = 0.5, alternative='greater') # p-value from binomial test for D1
    D2pval = stats.binom_test(D2first, n=total, p = 0.5, alternative='greater') # p-value from binomial test for D2
    # Record results
    pair_result = []
    pair_result.append(bnf1)
    pair_result.append(bnf2)
    pair_result.append(D1first)
    pair_result.append(D2first)
    pair_result.append(total)
    pair_result.append(D1pval)
    pair_result.append(D2pval)
    dir_result.append(pair_result)
    print(pair_result)

# pd.DataFrame(dir_result).to_csv('BNFChaptersDirTestResults.csv', index = False)   